In [ ]:
# %%capture --no-stderr

# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# %pip install -qU "langchain[openai]"
# %pip install -qU langchain-openai
# %pip install -qU langchain-core
# %pip install --upgrade --quiet langgraph langchain-community beautifulsoup4
# %pip install gradio
# %pip install langchain-chroma
# %pip install pypdf

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_CHAT_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_CHAT_API_VERSION"],
)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDINGS_API_VERSION"],
)

In [ ]:
# from langchain_core.vectorstores import InMemoryVectorStore

from langchain_community.vectorstores import Chroma
policy_vector_store = Chroma(persist_directory="./policy_vector_store", embedding_function=embeddings)
terminology_vector_store = Chroma(persist_directory="./terminology_vector_store", embedding_function=embeddings)

In [ ]:
# LOAD CONTEXT

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool

graph_builder = StateGraph(MessagesState)

@tool(response_format="content_and_artifact")
def retrieve_policy(query: str):
    """Use this tool when you need to check compliance of a feature
    against official regulations and policies."""
    retrieved_docs = policy_vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

@tool(response_format="content_and_artifact")
def retrieve_terminology(query: str):
    """Use this tool when the user query or context contains acronyms or terms
    that need clarification before checking compliance."""
    retrieved_docs = terminology_vector_store.similarity_search(query, k=10)
    serialized = "\n".join(
        f"{doc.metadata['term']}: {doc.page_content}"
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve_policy])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve_policy])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
    """
        You are an assistant for feature compliance checking. 
        Use the following pieces of retrieved context to flag
        whether this feature needs geo-specific compliance logic
        with clear reasoning. Optionally, provide the related 
        regulations. If you don't know the answer, say that you
        don't know. Use three sentences maximum and keep the
        answer concise.
        \n\n
    """
    f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [ ]:
# BUILD AND COMPILE GRAPH

from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent



# graph_builder.add_node(query_or_respond)
# graph_builder.add_node(tools)
# graph_builder.add_node(generate)

# graph_builder.set_entry_point("query_or_respond")
# graph_builder.add_conditional_edges(
#     "query_or_respond",
#     tools_condition,
#     {END: END, "tools": "tools"},
# )
# graph_builder.add_edge("tools", "generate")
# graph_builder.add_edge("generate", END)

memory = MemorySaver()
agent_executor = create_react_agent(llm, [retrieve_policy, retrieve_terminology], checkpointer=memory)
# graph = graph_builder.compile(checkpointer=memory)
# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}


In [ ]:
# # TEST INPUT

# input_message = "Does the following feature comply with the regulations stated? Feature: Universal PF deactivation on guest mode. Description: By default, PF will be turned off for all uses browsing in guest mode."

# for step in graph.stream(
#     {"messages": [{"role": "user", "content": input_message}]},
#     stream_mode="values",
#     config=config,
# ):
#     step["messages"][-1].pretty_print()
    

In [ ]:
import gradio as gr

def rag_chat(user_message, history):
    responses = []
    old_step_messages = []
    for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": user_message}]},
        stream_mode="values",
        config=config,
    ):
        if "messages" in step:
            responses.append(step["messages"][-1].content)

            # For debugging
            message_count_diff = len(step["messages"]) - len(old_step_messages)
            if message_count_diff > 0:
                new_messages = step["messages"][-message_count_diff:]
                for msg in new_messages:
                    msg.pretty_print()
                old_step_messages = step["messages"]

    reply = responses[-1] if responses else "No response."
    return reply

demo = gr.ChatInterface(rag_chat, title="Trivali Feature Compliance Checker")
demo.launch()
